# 앙상블 성능비교 - Voting vs Stacking

- 일반화 성능이 어느정도 인지 확인 해보자.
- 배송 물류가 제 시간에 도착 하였는지를 분류하는 Train 데이터셋 사용

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('./extrafiles/Train.csv')
df

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10994,10995,A,Ship,4,1,252,5,medium,F,1,1538,1
10995,10996,B,Ship,4,1,232,5,medium,F,6,1247,0
10996,10997,C,Ship,5,4,242,5,low,F,4,1155,0
10997,10998,F,Ship,5,2,223,6,medium,M,2,1210,0


In [2]:
df.columns
# 'ID', 'Warehouse_block', 'Mode_of_Shipment', 'Customer_care_calls',
#        'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases',
#        'Product_importance', 'Gender', 'Discount_offered', 'Weight_in_gms',
#        'Reached.on.Time_Y.N'

Index(['ID', 'Warehouse_block', 'Mode_of_Shipment', 'Customer_care_calls',
       'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases',
       'Product_importance', 'Gender', 'Discount_offered', 'Weight_in_gms',
       'Reached.on.Time_Y.N'],
      dtype='object')

In [3]:
# X, y 데이터 나누기
X = df[['ID', 'Warehouse_block', 'Mode_of_Shipment', 'Customer_care_calls',
       'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases',
       'Product_importance', 'Gender', 'Discount_offered', 'Weight_in_gms']]
y = df['Reached.on.Time_Y.N']


In [4]:
# 이상값 및 결측치 확인
X.isna().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64

In [5]:
# 이상치 확인
X.describe()

,ID,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms
count,10999.00000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000
mean,5500.00000,4.054459,2.990545,210.196836,3.567597,13.373216,3634.016729
std,3175.28214,1.141490,1.413603,48.063272,1.522860,16.205527,1635.377251
min,1.00000,2.000000,1.000000,96.000000,2.000000,1.000000,1001.000000
25%,2750.50000,3.000000,2.000000,169.000000,3.000000,4.000000,1839.500000
50%,5500.00000,4.000000,3.000000,214.000000,3.000000,7.000000,4149.000000
75%,8249.50000,5.000000,4.000000,251.000000,4.000000,10.000000,5050.000000
max,10999.00000,7.000000,5.000000,310.000000,10.000000,65.000000,7846.000000


In [6]:
# 카테고리 데이터 연속형 데이터 분리
X_num = X[['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product', 
           'Prior_purchases', 'Discount_offered', 'Weight_in_gms']]
X_cat = X[['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender', ]]

# 더미변수화
X_cat_dummy = pd.get_dummies(X_cat)

# 스케일링
from sklearn.preprocessing import MinMaxScaler
X_num_scaled = MinMaxScaler().fit_transform(X_num)
X_num_scaled = pd.DataFrame(X_num_scaled, index=X_num.index, columns=X_num.columns)

X_temp = pd.concat([X_num_scaled, X_cat_dummy], axis=1)
X_temp

,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms,Warehouse_block_A,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Warehouse_block_F,Mode_of_Shipment_Flight,Mode_of_Shipment_Road,Mode_of_Shipment_Ship,Product_importance_high,Product_importance_low,Product_importance_medium,Gender_F,Gender_M
0,0.4,0.25,0.378505,0.125,0.671875,0.033893,0,0,0,1,0,1,0,0,0,1,0,1,0
1,0.4,1.00,0.560748,0.000,0.906250,0.304894,0,0,0,0,1,1,0,0,0,1,0,0,1
2,0.0,0.25,0.406542,0.250,0.734375,0.346676,1,0,0,0,0,1,0,0,0,1,0,0,1
3,0.2,0.50,0.373832,0.250,0.140625,0.025712,0,1,0,0,0,1,0,0,0,0,1,0,1
4,0.0,0.25,0.411215,0.125,0.703125,0.216654,0,0,1,0,0,1,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10994,0.4,0.00,0.728972,0.375,0.000000,0.078451,1,0,0,0,0,0,0,1,0,0,1,1,0
10995,0.4,0.00,0.635514,0.375,0.078125,0.035939,0,1,0,0,0,0,0,1,0,0,1,1,0
10996,0.6,0.75,0.682243,0.375,0.046875,0.022498,0,0,1,0,0,0,0,1,0,1,0,1,0
10997,0.6,0.25,0.593458,0.500,0.015625,0.030533,0,0,0,0,1,0,0,1,0,0,1,0,1


In [7]:
# train test 데이터 셋 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_temp, y, stratify=y)

print(y_train.mean())
print(y_test.mean())

0.5966783852588192
0.5967272727272728


In [11]:
# 모델적용
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier

# decleare estimators
estimators_voting = [('lr', LogisticRegression()), ('sv', SVC(probability=True)), ('rf', RandomForestClassifier())]
estimators_stacking = [('sv', SVC()), ('rf', RandomForestClassifier())]
                        
model_voting = VotingClassifier(estimators=estimators_voting, voting='soft').fit(X_train, y_train)
model_stacking = StackingClassifier(estimators=estimators_stacking, final_estimator=LogisticRegression()).fit(X_train, y_train)
model_logi = LogisticRegression().fit(X_train, y_train)

In [12]:
# 교차검증 - Stacking 이 가장 좋다.
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5, shuffle=True, random_state=333)
score_voting = cross_val_score(VotingClassifier(estimators=estimators_voting, voting='soft'), X_train, y_train)
score_stacking = cross_val_score(StackingClassifier(estimators=estimators_stacking, final_estimator=LogisticRegression()), X_train, y_train)
score_logistic = cross_val_score(LogisticRegression(), X_train, y_train)

print("cross validation score (voting : >)", score_voting.mean())
print("cross validation score (stacking : >)", score_stacking.mean())
print("cross validation score (logi : >)", score_logistic.mean())

cross validation score (voting : >) 0.6546247679952956
cross validation score (stacking : >) 0.6549895069555469
cross validation score (logi : >) 0.6414110296414723


In [13]:
# prediction 이후 값 확인 비교 - 앙상블은 학습 데이터에 과적합 되므로 참고 자료로 활용도가 떨어지는듯.
# 개별 알고리즘간 비교가 필요할 듯
pred_voting = model_voting.predict(X_test)
pred_stacking = model_stacking.predict(X_test)
pred_logi = model_logi.predict(X_test)


# 설명력 분류도 설명력이라고 하나?!?
score_voting = model_voting.score(X_test, y_test)
score_stacking = model_stacking.score(X_test, y_test)
score_logi = model_logi.score(X_test, y_test)

print("voting score : ", score_voting)
print("stacking score : ", score_stacking)
print("logi score : ", score_logi)

# roc_auc_score
from sklearn.metrics import roc_auc_score
score_voting = roc_auc_score(y_test, pred_voting)
score_stacking = roc_auc_score(y_test, pred_stacking)
score_logi = roc_auc_score(y_test, pred_logi)

print("voting score : ", score_voting)
print("stacking score : ", score_stacking)
print("logi score : ", score_logi)

voting score :  0.6527272727272727
stacking score :  0.6512727272727272
logi score :  0.6378181818181818
voting score :  0.651137801676934
stacking score :  0.6483112245991332
logi score :  0.6285600776759206


In [14]:
# XGBoost는 어떨까?
# XGBClassifier
from xgboost import XGBClassifier
model_xgb = XGBClassifier().fit(X_train, y_train)

# 교차검증
score = cross_val_score(XGBClassifier(), X_train, y_train, cv=kfold)
print("XGB Classifier CV Score : >", score)

/Users/daehwajeong/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:03:12] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1637426408905/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/daehwajeong/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:03:14] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1637426408905/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/daehwajeong/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[00:03:14] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1637426408905/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/daehwajeong/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:03:15] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1637426408905/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/daehwajeong/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:03:15] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1637426408905/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGB Classifier CV Score : > [0.66484848 0.62727273 0.66060606 0.66484848 0.65979381]


In [15]:
score.mean()

0.6554739144017494

In [16]:
model_xgb.predict_proba(X_test)

array([[0.66815114, 0.3318489 ],
       [0.6662468 , 0.33375323],
       [0.31914353, 0.68085647],
       ...,
       [0.5069175 , 0.49308252],
       [0.00100726, 0.99899274],
       [0.12733042, 0.8726696 ]], dtype=float32)